In [8]:
import numpy as np
from tqdm import tqdm

In [15]:
# A toy stochastic cost function
rng = np.random.default_rng(0)
def cost(p_vec):
    return sum([p**2 for p in p_vec]) + rng.normal(scale=0.1)

In [4]:
# parameter dimension
dp = 10

## Finite Differences Stochastic Approximation (FDSA)

## Simultaneous Perturbation Stochastic Approximation (SPSA)

## Nelder-Mead Method

In [12]:
from numpy.linalg import norm

# Arguments can also be adaptive: gamma=1+2/dp, rho=0.75-1/(2*dp), sigma=1-1/dp (Adaptive Nelder-Mead Simplex (ANMS) method)
def nelder_mead(cost, dp, alpha=1, gamma=2, rho=0.5, sigma=0.5, nu=0.0001, max_iter=10000):
    # Initial simplex
    p_lst = [np.ones(dp)]
    for i in range(dp):
        temp = np.array(p_lst[0])
        temp[i] += 1
        p_lst.append(np.array(temp))
    cost_lst = [cost(p_vec) for p_vec in p_lst]

    for _ in range(max_iter):
        # Order
        cost_lst, p_lst = zip(*sorted(zip(cost_lst, p_lst))) # sort p_vec according to the cost function (ascending)
        cost_lst = list(cost_lst)
        p_lst = list(p_lst)

        # Termination test
        # Original termination criterion is based on the standard deviation of function values
        # However, serious problems may arise with stochastic functions
        # Another criterion is simplex size, ref: https://pubsonline.informs.org/doi/epdf/10.1287/mnsc.42.7.954
        dist_lst = [norm(p - p_lst[0]) for p in p_lst[1:]]
        if max(dist_lst) / max(1, norm(p_lst[0])) <= nu: # nu is the termination simplex size
            break

        # Centroid
        p_o = sum(p_lst[:-1]) / dp # centroid of all points except the last one

        # Reflection
        p_r = p_o + alpha * (p_o - p_lst[-1]) # alpha > 0
        cost_r = cost(p_r)
        if cost_lst[0] <= cost_r and cost_r < cost_lst[-2]:
            p_lst[-1] = p_r
            cost_lst[-1] = cost_r
            continue
        
        # Expansion
        elif cost_r < cost_lst[0]:
            p_e = p_o + gamma * (p_r - p_o) # gamma > 1
            cost_e = cost(p_e)
            if cost_e < cost_r:
                p_lst[-1] = p_e
                cost_lst[-1] = cost_e
                continue
            else:
                p_lst[-1] = p_r
                cost_lst[-1] = cost_r
                continue
        
        # Contraction
        else: # cost_lst[-2] <= cost_r
            p_c = p_o + rho * (p_r - p_o) # 0 < rho <= 0.5
            cost_c = cost(p_c)
            if cost_r < cost_lst[-1]:
                if cost_c < cost_r:
                    p_lst[-1] = p_c
                    cost_lst[-1] = cost_c
                    continue
            else: # cost_lst[-1] <= cost_r
                if cost_c < cost_lst[-1]:
                    p_lst[-1] = p_c
                    cost_lst[-1] = cost_c
                    continue
        
        # Shrink
        # Rare case, takes a lot of cost function evaluations, possibly omittable
        p_lst = [p_lst[0] + sigma * (p_vec - p_lst[0]) for p_vec in p_lst] # 0 < sigma <= 0.5
        cost_lst = [cost(p_vec) for p_vec in p_lst]
    
    else:
        # Optimization failed, number of iteration exceeds maximum value
        return
    
    return p_lst[0], cost_lst[0]

In [16]:
nelder_mead(cost, dp)

(array([-0.17608942, -0.06837984,  0.05445364,  0.06902426,  0.12681607,
         0.00848597, -0.07260046, -0.00766897,  0.11866098,  0.05102829]),
 -0.06227114678070228)

In [17]:
nelder_mead(cost, dp, gamma=1+2/dp, rho=0.75-1/(2*dp), sigma=1-1/dp)

(array([ 0.08012604, -0.03076585,  0.15595906, -0.02673476, -0.0604248 ,
        -0.00130607,  0.16061132,  0.175166  ,  0.03084868,  0.07864928]),
 -0.10352013119155362)